# Fileinput Exploration with Pandas and Plotly

This notebook was contributed to the Panel Gallery by [Marc Skov Madsen](https://datamodelsanalytics.com/) as an answer to the question [A button to upload csv file and plot it ](https://discourse.holoviz.org/t/a-button-to-upload-csv-file-and-plot-it/365) on [Discourse](https://discourse.holoviz.org/)

In Panel the File upload widget is called `Fileinput` and you can find the reference example [here](https://panel.holoviz.org/reference/widgets/FileInput.html#widgets-gallery-fileinput)

In [ ]:
import param
import panel as pn
import pandas as pd
import random
from datetime import datetime, timedelta
import io
import plotly.express as px

pn.extension('plotly')

Lets start out by creating some sample data

In [ ]:
class SampleDataApp(param.Parameterized):
    samples = param.Integer(default=40, bounds=(0,100))
    voltage_bounds=param.Range(default=(0,100), bounds=(0,1000))
    time_bounds=param.CalendarDateRange(
        default=(datetime(2020,2,1), datetime(2020,2,26)),
        bounds=(datetime(2020,1,1), datetime(2020,3,26)),
    )
    fub_ids = param.List(default=["a1","b1","b2"])
    
    sample_df = param.DataFrame()
    generate_sample_df = param.Action()
    
    file_name = param.String(default="sample_data.csv")
    save_sample_df = param.Action()
    
    def __init__(self, **params):
        super().__init__(**params)
        
        self.set_sample_df()
        
        self.generate_sample_df = self.set_sample_df
        self.save_sample_df = self.save_sample_data
        
    def set_sample_df(self, event=None):
        start = self.time_bounds[0]
        display(start)
        end = self.time_bounds[1]
        days = (end-start).days
        
        sample_data = {
            "Time": [start+timedelta(days=random.uniform(0,days)) for _ in range(0,self.samples)],
            "Voltage": [random.uniform(*self.voltage_bounds) for _ in range(0,self.samples)],
            "FubId": [random.choice(self.fub_ids) for _ in range(0,self.samples)],
        }
        self.sample_df = pd.DataFrame(sample_data) 
        
    
    def save_sample_data(self, event=None):
        if not self.sample_df is None:
            self.sample_df.to_csv(self.file_name, index=False)
            
    def view(self):
        return pn.Param(self)

sample_data_app = SampleDataApp()
sample_data_app.view()

**Click the `Save sample df` button**

Lets test the data can be read

In [ ]:
sample_df = pd.read_csv(sample_data_app.file_name, parse_dates=["Time"])
sample_df.head()

Let define our `VoltageApp`

In [ ]:
class VoltageApp(param.Parameterized):
    data = param.DataFrame()
    file_input = param.Parameter()
    show = param.Action()
    
    def __init__(self, **params):
        self.param.file_input.default = pn.widgets.FileInput()
        
        super().__init__(**params)
        
        self.plot_pane = pn.pane.Plotly(height=600, sizing_mode="stretch_width")
        self.show = self._show
                
    
    @param.depends("file_input.value", watch=True)
    def _parse_file_input(self):
        value = self.file_input.value
        string_io = io.StringIO(value.decode("utf8"))
        self.data=pd.read_csv(string_io, parse_dates=["Time"])
        
    def get_plot(self, df):
        assert ("Voltage" in df.columns) and ("Time" in df.columns), "no columns voltage and time"
        df = (df.loc[df['Voltage'] != 'Invalid/Calib']).copy(deep=True)
        df['Voltage'] = df['Voltage'].apply(lambda x: float(x))
        df.reset_index()

        traces = []

        if "FubId" in df.columns:
           return px.scatter(df, x="Time", y="Voltage", color="FubId")

        return px.scatter(df, x="Time", y="Voltage")
    
    @param.depends("data", watch=True)
    def _set_plot(self):
        self.plot_pane.object=self.get_plot(self.data)
        
    def _show(self, event=None):
        pn.Column(
            self.file_input,
            self.plot_pane,
        ).show()
        
    def view(self):
        return pn.Column(
            self.file_input,
            self.plot_pane,
            self.param.show,
        )
    
voltage_app = VoltageApp()
voltage_app.view()

## Todo

- Solve bug: https://github.com/holoviz/panel/issues/1195